<img width="8%" alt="Google Maps.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Maps.png" style="border-radius: 15%">

# Google Maps - Calculate travel costs between two addresses
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Maps+-+Calculate+travel+costs+between+two+addresses:+Error+short+description">Bug report</a>

**Tags:** #googlemaps #productivity #operations #automation #jupyternotebooks

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Last update:** 2023-07-26 (Created: 2023-07-25)

**Description:** This notebook calculates the travel costs between two addresses using Google Maps API.

## Input

### Import libraries

In [7]:
import os
import requests
import naas

### Setup variables
Pre-requisite: 
1. Follow the steps in the link - https://developers.google.com/maps/documentation/routes/cloud-setup
2. Sign up for an account with the Routes API provider.
3. Make sure to enable Routes API as it is in the "Additional APIs" section in the google cloud console.
4. Check the available countries https://developers.google.com/maps/documentation/routes/calculate_toll_fees#toll-locations

- `api_key`: This variable holds the Google Cloud Platform (GCP) API key. The key is retrieved from the secret variable using the Naas secret manager.
- `start_location`: This variable represents the starting location for a journey. In this case, it's set to 'San Francisco, CA'.
- `end_location`: This variable indicates the end location or destination for a journey. Here, it's set to 'Los Angeles, CA'.

In [18]:
api_key = naas.secret.get("GCP_MAP_API_KEY")  # Read API key from the secret variable
start_location = 'San Francisco, CA'
end_location = 'Los Angeles, CA'

## Model

### Get coordinates from address

In [19]:
def get_coordinates(api_key, address):
    # Init
    latitude = None
    longitude = None
    
    # API endpoint
    api_endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'

    # Setup params
    params = {
        'address': address,
        'key': api_key
    }
    response = requests.get(api_endpoint, params=params)

    # Response
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract and utilize the geolocation information from the response
            results = data['results']
            for result in results:
                latitude = result['geometry']['location']['lat']
                longitude = result['geometry']['location']['lng']
                print("📍 Address:", address)
                print(f'🗺️ Latitude: {latitude}, Longitude: {longitude}')
        else:
            print('Geocoding request failed.')
    else:
        print('Geocoding request failed with status code:', response.status_code)
    return latitude, longitude

print("Origin:")
origin_lat, origin_lng = get_coordinates(api_key, start_location)
print()
print("Destination:")
destination_lat, destination_lng = get_coordinates(api_key, end_location)

## Output

### Calculate travel costs between two addresses
Request for toll information (more info on https://developers.google.com/maps/documentation/routes/calculate_toll_fees)

In [27]:
api_endpoint = 'https://routes.googleapis.com/directions/v2:computeRoutes'

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': api_key,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.travelAdvisory.tollInfo,routes.legs.travelAdvisory.tollInfo'
}

payload = {
    "origin": {
        "location": {
            "latLng": {
                "latitude": origin_lat,
                "longitude": origin_lng
            }
        }
    },
    "destination": {
        "location": {
            "latLng": {
                "latitude": destination_lat,
                "longitude": destination_lng
            }
        }
    },
    "travelMode": "DRIVE",
    "extraComputations": ["TOLLS"],
    "routeModifiers": {
        "vehicleInfo": {
            "emissionType": "GASOLINE"
        },
        "tollPasses": [
           # Add relevant toll passes if applicable
        ]
    }
}

response = requests.post(api_endpoint, json=payload, headers=headers)

# Manage response
if response.status_code == 200:
    data = response.json()
    routes = data['routes']
    if routes:
        distance_meters = routes[0]['distanceMeters']
        duration = routes[0]['duration']
        print("Distance in Km:", round(distance_meters / 1000, 2))
        print("Duration in Hours:", round(int(duration[:-1]) / 3600, 2))
        if 'travelAdvisory' in routes[0]:
            toll_info = routes[0]['travelAdvisory']['tollInfo']
            estimated_price = toll_info['estimatedPrice']
            if estimated_price:
                currency_code = estimated_price[0]['currencyCode']
                toll_cost = float(estimated_price[0]['units'])
                formatted_toll_cost = '{:.2f}'.format(toll_cost)
                print('Toll Cost:', currency_code + ' '+ formatted_toll_cost)
    else:
        print('No routes found in the response.')
else:
    print('Google Maps API request failed with status code:', response.status_code)